# Single-cell RNA-seq QC and Preprocessing Pipeline - 30.06.2025

## Amaç:
- Bu script, single-cell RNA-seq verilerinin kalite kontrolü (QC), filtreleme ve ilk ön işleme adımlarını içerir. 

## İçerik:
- Gerekli paketlerin kurulumu ve çağrılması
- PBMC (Peripheral Blood Mononuclear Cells) veri setinin yüklenmesi
- QC metriklerinin hesaplanması ve görselleştirilmesi
- Hücre filtrasyonu
- Son kontrol plotları

## Kullanılan veri seti:
- 10X Genomics PBMC (10k_PBMC.h5)

## 1) Paket Kurulum ve Çağırma

### CRAN üzerinden kurulan paketler

- install.packages("Seurat")
- install.packages("dplyr")
- install.packages("patchwork")
- install.packages("ggplot2")
- install.packages("hdf5r") # .h5 formatındaki 10X Genomics veri dosyalarını okuyabilmek için gerekli
- install.packages("BiocManager")

### Bioconductor üzerinden kurulan paketler
- BiocManager::install("clusterProfiler")
- BiocManager::install("limma")
- BiocManager::install("org.Hs.eg.db")
- BiocManager::install("celldex")
- BiocManager::install("SingleR")
- BiocManager::install("scrapper")

## Paket Kurulum ve Çağırma Farkı
- **install.packages() veya BiocManager::install()**
    - Paketi bir kereye mahsus bilgisayara indirir ve kurar.
- **library()**
    - Paketi proje veya oturum içinde aktif hale getirir.
    - Her oturumda tekrar çağrılması gerekir.

## Organizmaya özel veri tabanları
- org.Hs.eg.db: Homo sapiens (insan) genom veri tabanı.
- org.Mm.eg.db: Mus musculus (fare) genom veri tabanı.


## 2) Projede kullanılacak kütüphaneleri çağırma

- library(Seurat)
- library(dplyr)
- library(patchwork)
- library(ggplot2)
- library(hdf5r)
- library(clusterProfiler)
- library(celldex)
- library(org.Hs.eg.db)
- library(SingleR)
- library(scrapper)

## 3) PBMC Dataset Yükleme ve Seurat Nesnesi Oluşturma
### KOD:
- **pbmc.data <- Read10X_h5("10k_PBMC.h5")**
- **pbmc <- CreateSeuratObject(counts = pbmc.data, project = "pbmc10k", min.cells = 3, min.features = 200)**
- **pbmc**

### Veriyi yükleme
- Load the PBMC dataset (10X Genomics formatındaki .h5 dosyası)
> **pbmc.data <- Read10X_h5("10k_PBMC.h5")**

### Load the PBMC dataset (10X Genomics formatındaki .h5 dosyası)
> **pbmc.data <- Read10X_h5("10k_PBMC.h5")**
- PBMC (Peripheral Blood Mononuclear Cells): Kan örneklerinden izole edilen mononükleer hücreler (lenfositler, monositler vb.).
- .h5: 10X Genomics tarafından kullanılan HDF5 formatındaki veri dosyası.

### Seurat objesi oluşturma
> **pbmc <- CreateSeuratObject(counts = pbmc.data, 
                           project = "pbmc10k", 
                           min.cells = 3, 
                           min.features = 200)**

### Parametre açıklamaları
- counts = pbmc.data: Ham count (sayı) matrisi.
- project = "pbmc10k": Proje etiketi (meta veri olarak tutulur).
- min.cells = 3:
    -  Bir gen en az 3 hücrede ifade edilmiş olmalı, yoksa filtrelenir (az görülen gürültü genler temizlenir).
    min.features = 200:
    - Bir hücrede en az 200 farklı gen tespit edilmeli, yoksa o hücre atılır (ölü hücre veya düşük kalite hücreler temizlenir).


## 4) Mitokondriyel Gen Yüzdesi (percent.mt) Hesaplama
### KOD:
- **pbmc[["percent.mt"]] <- PercentageFeatureSet(pbmc, pattern = "^MT-")**

### percent.mt nedir?
- pattern = "^MT-" → Gen isimlerinin başında "MT-" arar.
- "MT-" → Mitokondriyel genleri ifade eder (örn: MT-CO1, MT-ND2).
- Mitokondriyel gen yüzdesi, her hücrede mitokondri kaynaklı genlerin toplam RNA içindeki oranını gösterir.

### Neden önemli?
- Eğer bir hücrede mitokondriyel gen oranı çok yüksekse, bu genellikle:
- Hücrenin ölmekte olduğunu,
- Stres altında veya patlamış (lizis olmuş) olduğunu gösterir.
- Bu hücreler genelde filtrelenir, çünkü biyolojik anlamlı veri üretmez.


##  5) İlk 5 hücre için QC metriklerini görüntüleme
### KOD:
- **head(pbmc@meta.data, 5)**

### Ne yapıyor?
- pbmc@meta.data: Seurat objesinin metadata tablosu.
- Bu tabloda her hücreye ait:
- nFeature_RNA → O hücrede tespit edilen gen sayısı.
- nCount_RNA → O hücrede toplam RNA molekülü sayısı (UMI sayısı).
- percent.mt → Mitokondriyel gen yüzdesi.
- (Veya eklediğin başka QC kolonları.)

### Amaç
- Hücrelerin kalite kontrol (QC) metriklerini hızlıca kontrol etmek.
- Örn: Mitokondri yüzdesi çok yüksek hücre var mı?
- Veri ön filtreleme adımlarında fikir verir.

## 6) Kalite Kontrol (QC) Metriklerinin Violin Plot ile Görselleştirilmesi

### KOD:
- **VlnPlot(pbmc, 
        features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), 
        ncol = 3, 
        pt.size = 0.0001)**

### Parametre açıklamaları
- features = c(...)
-  Gösterilecek QC metrikleri:
    - nFeature_RNA: Hücre başına tespit edilen gen sayısı.
    - nCount_RNA: Toplam RNA molekülü (UMI sayısı).
    - percent.mt: Mitokondriyel gen oranı.
- ncol = 3
    - 3 sütun şeklinde yan yana gösterir.
- pt.size = 0.0001
    - Hücreleri temsil eden noktaların boyutunu çok küçük yapar (yoğunluğu görmek için).

### Her metrik neyi gösteriyor?
- nFeature_RNA → Hücrede kaç farklı gen var (çeşitlilik).
- nCount_RNA → Hücrede toplam kaç RNA okunmuş (yoğunluk).
- percent.mt → Mitokondriyel RNA yüzdesi (hücre kalitesi).

### Nasıl filtrelenir?
- nFeature_RNA ve nCount_RNA için:
    - Alttan yatay çizgiyle kes → Düşük kalite (ölü veya boş) hücreleri at.
    - Üst kısmı (kaliteli olanları) al.
- percent.mt için:
    - Alttan kes → Mitokondri oranı yüksek olan problemli hücreleri at.
    - Alt kısmı (sağlıklı olanları) al.

### Amaç
- Hücre kalitesini hızlıca değerlendirmek.
- Hangi hücrelerin filtreleneceğine karar vermek.


## 7) FeatureScatter ile QC metriklerinin karşılaştırılması
### KOD:
- **plot1 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "percent.mt") +**
  **theme(legend.position="none")**
- **plot2 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA") +**
  **theme(legend.position="none")**
- **plot1 + plot2**

### Ne yapar bu kod?
**FeatureScatter:**
- İki metrik arasındaki ilişkiyi dağılım grafiği (scatter plot) olarak gösterir.
- feature1 ve feature2 olarak belirtilen metriklerin hücre bazında korelasyonunu görselleştirir.

### plot1: nCount_RNA vs percent.mt
- Her nokta = bir hücre.
- X ekseni: Toplam RNA sayısı (nCount_RNA)
- Y ekseni: Mitokondriyel oran (percent.mt)
- Amaç: Çok fazla RNA içeren hücrelerde mitokondri oranı da yüksek mi?

### plot2: nCount_RNA vs nFeature_RNA
- X ekseni: Toplam RNA sayısı
- Y ekseni: Farklı gen sayısı
- Amaç: Çok RNA içeren hücreler aynı zamanda çeşitli genler mi barındırıyor?

### Neden önemli?
- Bu grafikler sayesinde:
    - Çıktılardaki outlier (aykırı) hücreleri görüp filtreleme kararları alabilirsin.
    - Örneğin:
    - Aşırı yüksek nCount ama düşük nFeature → çift hücre (doublet) olabilir.
    - Yüksek percent.mt → ölü veya hasarlı hücre olabilir.



## Korelasyon: -1 ve 1 arasında değişiyor:
- 0 'da hiç yok demek/0'a yakınsa bağlantı yok korelasyon yok
- 1'e yakında 1'i artarken diğeride artar
- -1'e yakında bir tanesi artarken diğeri azalır

## 8) Hücre Filtreleme (Subset) Adımı
### KOD:
- **pbmc <- subset(pbmc, subset = nFeature_RNA > 200 & nFeature_RNA < 5000 & percent.mt < 15)**


## Ne yapıyor?
- **subset()** fonksiyonu, hücreleri kalite kriterlerine göre filtreler.
- Filtreleme kriterleri, subset = içinde belirtilir.
- **nFeature_RNA > 200** :	Bir hücrede en az 200 farklı gen bulunmalı (ölü veya düşük kalite hücreler atılır).
- **nFeature_RNA < 5000** :	Çok fazla gen ifadesi olan hücreler atılır (ör: doublet — iki hücre karışmış olabilir).
- **percent.mt < 15** :	Mitokondriyel RNA oranı %15’ten küçük olmalı (yüksek mt → ölü veya hasarlı hücre).

## 9) Violin Plot ile QC Metriklerinin Son Kontrolü
### KOD:
- **VlnPlot(pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3, pt.size = 0.001)**

### Parametre açıklamaları
- **features = c(...)**
    - Gösterilecek metrikler:
- **nFeature_RNA:** Hücre başına farklı gen sayısı.
- **nCount_RNA:** Toplam RNA (UMI) sayısı.
- **percent.mt:** Mitokondriyel RNA oranı.
- ncol = 3
    - Grafikleri 3 sütun halinde yan yana diz.
- pt.size = 0.001
    - Noktaların boyutu; her nokta bir hücreyi temsil eder. Küçük seçilerek yoğunluk daha iyi gösterilir.

### Amaç
- Filtreleme işleminden sonra hücre kalitesini görsel olarak kontrol etmek.
- Aykırı hücrelerin (outlier) veya istenmeyen dağılımların kalıp kalmadığını incelemek.

### Yorum
- nFeature_RNA ve nCount_RNA için:
    - Genelde alttan kesilir → düşük kaliteli hücreler atılır.
- percent.mt için:
    - Üstten kesilir → mitokondri oranı yüksek hücreler atılır.


## 10)Veri Normalizasyonu
### KOD:
- **pbmc <- NormalizeData(pbmc, 
                      normalization.method = "LogNormalize", 
                      scale.factor = 10000, 
                      verbose = FALSE)**

### Ne yapar?

- NormalizeData() → Her hücrede toplam RNA (UMI) sayısını normalize eder.
- **normalization.method = "LogNormalize"**
- Her hücredeki gen sayıları toplam RNA miktarına bölünür.
- Sonra 10,000 (scale.factor) ile çarpılır.
- Son adımda log1p (log(x+1)) alınır.

### Parametreler
- Parametre	Açıklama
- **normalization.method** -> Default: "LogNormalize". Hücresel kütle etkilerini düzeltir.
- **scale.factor** -> Default: 10,000. Her hücreyi aynı ölçeğe getirir.
- **verbose** -> TRUE yaparsan, adım adım ilerleme çıktısı gösterir.

### Notlar
- Parametreleri yazmak zorunda değilsin; hepsinin default değerleri vardır.
- verbose = TRUE → Kodun nasıl çalıştığını, hangi hücrede ne yaptığını görmek için faydalıdır.


## 11) Değişken Genlerin Seçilmesi (Highly Variable Genes)
### KOD:

- **pbmc <- FindVariableFeatures(pbmc, 
                             selection.method = "vst", 
                             nfeatures = 2000, 
                             verbose = FALSE)**

### Ne yapar?
- FindVariableFeatures(), en çok varyans gösteren genleri (highly variable genes, HVGs) seçer.
- Bu genler:
- Hücreler arası biyolojik farklılıkları daha iyi gösterir.
- Boyut indirgeme (PCA, UMAP vb.) ve cluster analizleri için kullanılır.
- Parametreler
    - **selection.method**:	"vst" (Varsayılan) → Variance Stabilizing Transformation; gürültüyü azaltır, biyolojik sinyali korur.
    - **nfeatures**:	Seçilecek gen sayısı (default: 2000). Analize göre değiştirilebilir.
    - **verbose**:	TRUE yaparsan, adım adım hangi genlerin seçildiğini gösterir.

### Neden önemli?
- Çok fazla gen analize dahil edilirse, gürültü artar.
- Sadece değişken genler alınarak:
- Hesap yükü azalır.
- Daha anlamlı biyolojik örüntüler ortaya çıkar.

## 12) En Değişken 10 Genin Belirlenmesi

### KOD:
- **top10 <- head(VariableFeatures(pbmc), 10)**

### Ne yapar?

- VariableFeatures(pbmc)
    - Seurat objesinde seçilen tüm değişken genlerin (highly variable genes) listesini döndürür.
    head(..., 10)
    - Bu listenin ilk 10 genini alır (en yüksek varyansa sahip olanlar).

### Amaç
- En değişken genleri hızlıca görmek.
- Marker gen taraması, örnek görselleştirme veya downstream analizlerde odaklanmak için kullanılabilir.


## 13) Değişken Genlerin Görselleştirilmesi (Highly Variable Genes)
### KOD:
- **plot1 <- VariableFeaturePlot(pbmc) + 
  theme(legend.position = "top")**
- **plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE) + 
  theme(legend.position = "none")**
- **plot1 + plot2**


### Ne yapar?

- **VariableFeaturePlot()**
- Seçilen değişken genlerin (variable features) dağılımını gösterir.
- Y ekseni: Genin varyansı.
- X ekseni: Ortalama ifadesi (mean expression).
- En değişken genler grafikte üst kısımlarda görünür.

- **LabelPoints()**
- Grafik üzerine seçilen gen isimlerini etiket (label) olarak ekler.
- points = top10: En değişken 10 geni etiketle.
- repel = TRUE: Etiketlerin üst üste binmesini engeller.

### Amaç
- En değişken genleri vurgulamak.
- Veri setinde hangi genlerin ayırt edici olduğunu kolayca görmek.

### Parametre açıklamaları
- points	Etiketlenecek gen listesi.
- repel = TRUE	Etiketlerin çakışmasını önler, daha okunabilir yapar.

### Sonuç
- plot1 → Değişken genlerin dağılımı, etiketsiz.
- plot2 → En değişken 10 gen etiketlenmiş hali.
- plot1 + plot2 → Yan yana iki grafik.


## Değişken Genler Scatter Plot Yorumu

### Her nokta neyi temsil eder?

- Her nokta = 1 gen.
- Noktalar, genlerin ifade düzeylerine (average expression) ve varyanslarına göre konumlandırılır.

#### X ekseni: Average Expression
- Genin ortalama ifade seviyesi.
- Yani gen genel olarak ne kadar "aktif", ne kadar yüksek miktarda RNA var?

#### Y ekseni: Varyans (Dispersion)
- Genin hücreler arasında ne kadar değişken olduğunu gösterir.
- Daha yüksek varyans → Hücreler arası daha fazla farklılık → Daha bilgilendirici.

### Noktaların konumu ne anlama gelir?
- Sağda ve yukarıda olan noktalar:
- Hem yüksek ortalama ifadeye sahip,
- Hem de yüksek varyans gösteriyor.
- Bu genler genellikle biyolojik olarak daha anlamlı ve ayırt edici.


## 14) Veri Ölçeklendirme (Scaling)
### KOD:
- **all.genes <- rownames(pbmc)**

### Ne yapar?
- **rownames(pbmc)**:
    - pbmc nesnesindeki tüm genlerin isimlerini döndürür.
 
### KOD:
- **pbmc <- ScaleData(pbmc, features = all.genes, verbose = FALSE)**

### ScaleData() ne yapar?
- Her gen için:
    - Ortalamayı 0 yapar.
    - Varyansı 1 olacak şekilde standardize eder.
- Böylece:
    - Genler farklı ölçekte olsa bile aynı temelde karşılaştırılabilir olur.
    - Boyut indirgeme (ör: PCA) ve clustering adımları daha sağlıklı yapılır.
- features = all.genes → Tüm genlere ölçeklendirme uygulanır.
- Sadece değişken genleri kullanmak da mümkündür (daha hızlı olur), ama burada tüm genler seçilmiş.


## 15)  PCA (Principal Component Analysis) Uygulama
- **pbmc <- RunPCA(pbmc, features = VariableFeatures(object = pbmc), verbose = FALSE)**


### Ne yapar?

- PCA (Principal Component Analysis):
- Yüksek boyutlu veriyi (örn: binlerce gen) daha az boyuta indirger.
- En fazla varyansı açıklayan yeni eksenler (principal components) oluşturur.
- Burada:
- features = VariableFeatures(object = pbmc)
    - Sadece daha önce seçilen değişken genler PCA’ya dahil edilir.

### Neden?
- Veride binlerce gen (özellik) → Yani çok yüksek boyut.
- Hücre sayısı kadar boyut var gibi düşünebilirsin.
- Biz insanlar 3 boyutlu görselleştirme yapabiliyoruz (ör: scatter plot).
- PCA sayesinde:
- Karmaşık yapı özetlenir.
- Cluster yapıları ve hücre tipleri daha iyi ayrışır.
### Özet
- Veri: ~20.000 gen × binlerce hücre -> PCA: Boyut indirgeme → Örn: İlk 10–30 principal component -> Daha az gürültü, daha net biyolojik sinyal

### Not
- verbose = FALSE → Çalışma sırasında detaylı çıktı vermez.
- İstersen verbose = TRUE yazarak adım adım ilerlemeyi görebilirsin.

## 16)PCA Dağılım Plot'u (DimPlot)
### KOD:
- **DimPlot(pbmc, reduction = "pca")**

### Ne yapar?

- PCA sonucu elde edilen principal component'ları kullanarak hücreleri 2D (veya 3D) bir düzlemde gösterir.
- Her nokta = bir hücre.

### Boyut indirgeme mantığı

- Aslında veri, binlerce gen × binlerce hücre şeklinde çok boyutlu (yüksek boyutlu) bir uzayda bulunur.
- PCA (veya diğer teknikler) bu uzayı 2 veya 3 boyuta indirger, böylece:
- Görselleştirilebilir,
- Hücre dağılımı, örüntüler ve cluster yapıları daha kolay yorumlanır.

### Amaç
- Hücrelerin genel dağılımını görmek.
- Veri setinde doğal grup oluşumlarını (ör: potansiyel cluster’lar) fark etmek.
- Outlier veya aykırı hücreleri hızlıca fark etmek.

## 17)Elbow Plot (Dirsek Grafiği)
### KOD:
- **ElbowPlot(pbmc)**

### Ne yapar?
- PCA sonucu elde edilen principal component'ların (PC) her birinin varyansa katkısını gösterir.
- Yani her PC’nin ne kadar bilgi (varyans) açıkladığını görürüz.

### Amaç
- "Kolun dirseğini bulma" (elbow point):
- PC'lerin varyans açıklama katkısı hızlıca düştüğü noktayı belirlemek.
- O noktadan sonra eklenen PC'ler çoğunlukla gürültü (noise) taşır.
- Böylece:
- Kullanılacak ideal PC sayısını belirleriz.
- Genelde ~10–30 PC seçilir (veri setine bağlı olarak değişir).

### Özet Not
- Principal Components arttıkça:
    - Açıklanan varyans azalır.
    - Gürültü artar.
    - Dirsek noktası → "Yeterli bilgi, minimum gürültü".

### SORU GELİR
- Kaç PC kullanılmalı?
    - Cevap: Dirsek noktasına kadar olan PC'ler seçilir.
    - Grafik üzerinden interaktif yorum yapılır, sabit bir sayı yoktur.

## 18) Kümelenme (Clustering) ve UMAP Görselleştirme
### KOD:
- **pbmc <- FindNeighbors(pbmc, dims = 1:10, verbose = FALSE)**
- **pbmc <- FindClusters(pbmc, resolution = 0.5, verbose = FALSE)**
- **pbmc <- RunUMAP(pbmc, dims = 1:10, verbose = FALSE)**
- **DimPlot(pbmc, reduction = "umap")**

## Ne yapar?
- **FindNeighbors()**
    - KNN (K-en Yakın Komşu) algoritmasına dayalı komşuluk grafiği kurar.
    - dims = 1:10: PCA’dan seçilen ilk 10 bileşeni kullanır.
- **FindClusters()**
    - Hücreleri benzer gen ifadesine göre kümelere ayırır.
    - resolution parametresi:
        - Daha yüksek değer → Daha fazla, küçük kümeler.
        - Daha düşük değer → Daha az, büyük kümeler.
- **RunUMAP()**
    - UMAP (Uniform Manifold Approximation and Projection) boyut indirgeme tekniği.
    - Hücrelerin 2D (veya 3D) düzlemde daha anlamlı dağılmasını sağlar.
- **DimPlot()**
    - UMAP çıktısını görselleştirir.
    - Her nokta = bir hücre, renkler = kümeler (cluster’lar).

### Amaç
- Hücreleri gen ifadesine göre gruplara (cluster) ayırmak.
- Her cluster potansiyel bir hücre tipi temsil eder.

### Cluster yorumlama
- Bir küme içindeki hücreler:
    - Birbirine çok benzer gen ifadelerine sahip olmalı.
- Farklı kümeler:
    - Birbirlerinden farklı gen ifadesine sahip olmalı.
- Cluster’ları anlamak için:
    - O kümede diğerlerine göre daha yüksek ifade edilen genler (DEG) bulunur.
- Marker genler kullanılır:
    - Ör: Eğer bir kümede OCA1 geni yüksek → O küme olası melanocit kümesi.
    - Ör: CD3D geni yüksek → T hücre kümesi.

### Önemli Not
- dims = 1:10 dediysek,
    - Hem FindNeighbors, hem FindClusters, hem RunUMAP içinde aynı aralık kullanılmalı.
